In [1]:
import numpy as np

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column, gridplot

output_notebook(hide_banner=True)

# Méthode de Runge-Kutta

## La méthode RK2

**Implémenter la méthode RK2. Tester le cas $f(t,y)=y$, pour $t_0=0$, $y_0=1$ et $T=1,$.**

In [2]:
def update_rk2(f, t, y, h):
    """retourne l'approximation yn+1 de y au temps tn+1 = tn + h avec une méthode de Runge-Kutta d'ordre 2
    
    Parameters
    ----------
    f : fonction f(t,y) second membre de l'EDO
    t : temps tn
    y : valeur de yn
    h : pas de temps
    
    Returns
    -------
    ynp1 : valeur de yn+1
    """
    ynp1 = y + h*f( t+h/2, y + h/2*f(t,y) )
    return ynp1


def rk2(f, tini, tend, nt, yini):
    """retourne le vecteur contenant les valeurs des yn en appelant update_rk2
    
    Parameters
    ----------
    f : fonction f(t,y) second membre de l'EDO
    tini : temps initial
    tend : temps final
    nt : dimension du vecteur de sortie
    yini : valeur initiale y(tini)
    
    Returns
    -------
    y : vecteur les nt valeurs yn
    """

    t = np.linspace(tini, tend, nt)
    y = np.zeros(nt)
    # au cas où la dimension de yini est supérieure à 1 (exercice 3)
    if type(yini) == np.ndarray :
        y = np.zeros( (nt,len(yini)) )
    #----------------------------------
    y[0] = yini
    h = t[1]-t[0]
    
    for n in range(1,nt) :
        y[n] = update_rk2(f,t[n-1],y[n-1],h)
    
    return y

In [3]:
def f(t,y) :
    return y

In [4]:
tini = 0
tend = 10
nt = 50
yini = 1

# Approximation numérique de la solution de l'EDO en utilisant la méthode de RK2
y = rk2(f, tini, tend, nt, yini)
t = np.linspace(tini, tend, nt)

# La fonction exacte solution de l'EDO
tsol = np.linspace(tini, tend, 200)
ysol = np.exp(tsol)

# Figure
fig = figure(width=490, height=300)
fig.line(t,y, legend = "y_RK2")
fig.line(tsol,ysol, color="orange", legend = "exp(t)")
fig.legend.location = "top_left"

show(fig)

**Commentaire :**  
La solution générale de l'equation différentielle $y'=y$ est de la forme $y(t) = \alpha e^{t}$, et la condition initiale $y(0)=1$ impose d'avoir $\alpha=1$, donc $y(t) = e^{t}$  
On remarque que l'approximation obtenue par la méthode de Runge-Kuta donne une courbe fiable de la solution de cette equation différentielle.  

**Vérifier que l'erreur de convergence $\max_{0\leq n \leq N } |y(t_n) - y_n| = O(h^2)$, c'est à dire que la méthode converge à l'ordre 2.**

In [304]:
def max(x) :
    m = x[0]
    for i in range(len(x)) :
        if x[i]>m : m=x[i]
    return m

In [305]:
tini = 0
tend = 1
yini = 1

# le tableau "entiers" contient les valeurs de nt, et E l'erreur de convergence associées à chaque valeur
N = 200
ntini =10
pas = 10

entiers = np.array([n for n in range(ntini, N, pas)])
E = np.zeros(len(entiers))

# Remplissage de E
for i in range(len(entiers)) :
    nt = entiers[i]
    y = rk2(f, tini, tend, nt, yini)
    t = np.linspace(tini, tend, nt)
    ysol = np.exp(t)
    err = abs(ysol-y)
    E[i] = max(err)

# Figure
fig = figure(width=490, height=300, x_axis_type="log", y_axis_type="log")
C1 = E[0]*entiers[0];  C2 = E[0]*entiers[0]**2;  C3 = E[0]*entiers[0]**3
fig.line(entiers,C1/entiers, color="yellow", legend="ordre 1")
fig.line(entiers,C2/entiers**2, color="orange",legend="ordre 2")
fig.line(entiers,C3/entiers**3, color="pink", legend="ordre 3")
fig.x(entiers,E, legend = "max|y(t)-yn|_RK4")
fig.legend.location = "bottom_left"

show(fig)

**Commentaire :**  
J'ai représenté sur la figure ci-dessus chacun des ordres 1,2 et 3, et j'ai marqué les points où j'ai calculé l'erreur de convergence. 
On remarque alors que la convergence est d'ordre 2.  
**Remarque :** Les constantes C1,C2, et C3 que j'ai introduites dans mon programme servent uniquement à faire coincider toutes les courbes au premier point. Elles n'ont pas d'effet sur le résultat recherché car elles ne modifient pas les pentes.

## La méthode RK4

**Implémenter la méthode RK4 et vérifier numériquement sur le même exemple qu'à la question précédente.**

In [306]:
def update_rk4(f, t, y, h):
    """retourne l'approximation yn+1 de y au temps tn+1 = tn + h avec une méthode de Runge-Kutta d'ordre 4
    
    Parameters
    ----------
    f : fonction f(t,y) second membre de l'EDO
    t : temps tn
    y : valeur de yn
    h : pas de temps
    
    Returns
    -------
    ynp1 : valeur de yn+1
    """
    
    k1 = f(t,y)
    k2 = f(t+h/2, y+h/2*k1)
    k3 = f(t+h/2, y+h/2*k2)
    k4 = f(t+h, y+h*k3)
    ynp1 = y + h/6*(k1 + 2*k2 + 2*k3 + k4)
    return ynp1


def rk4(f, tini, tend, nt, yini):
    """retourne le vecteur contenant les valeurs des yn en appelant update_rk4
    
    Parameters
    ----------
    f : fonction f(t,y) second membre de l'EDO
    tini : temps initial
    tend : temps final
    nt : dimension du vecteur de sortie
    yini : valeur initiale y(tini)
    
    Returns
    -------
    y : vecteur les nt valeurs yn
    """
    
    t = np.linspace(tini, tend, nt)
    y = np.zeros(nt)
    # au cas où la dimension de yini est supérieure à 1 (exercice 3)
    if type(yini) == np.ndarray :
        y = np.zeros( (nt,len(yini)) )
    #----------------------------------
    y[0] = yini
    h = t[1]-t[0]
    
    for n in range(1,nt) :
        y[n] = update_rk4(f,t[n-1],y[n-1],h)
    
    return y

**Vérifier que cette méthode est d'ordre 4.**

In [307]:
tini = 0
tend = 1
yini = 1

# le tableau "entiers" contient les valeurs de nt, et E l'erreur de convergence associées à chaque valeur
N = 100
ntini =10
pas = 10

entiers = np.array([n for n in range(ntini, N, pas)])
E = np.zeros(len(entiers))

# Remplissage de E
for i in range(len(entiers)) :
    nt = entiers[i]
    y = rk4(f, tini, tend, nt, yini)
    t = np.linspace(tini, tend, nt)
    ysol = np.exp(t)
    err = abs(ysol-y)
    E[i] = max(err)

# Figure
fig = figure(width=490, height=300, x_axis_type="log", y_axis_type="log")
C1 = E[0]*entiers[0]**3;  C2 = E[0]*entiers[0]**4;  C3 = E[0]*entiers[0]**5
fig.line(entiers,C1/entiers**3, color="yellow", legend="ordre 3")
fig.line(entiers,C2/entiers**4, color="orange",legend="ordre 4")
fig.line(entiers,C3/entiers**5, color="pink", legend="ordre 5")
fig.x(entiers,E, legend = "max|y(t)-yn|_RK4")
fig.legend.location = "bottom_left"

show(fig)

**Commentaire :**  
On remarque que l'ordre de convergence est 4.  
**Remarque :** Les points obtenus sont (légèrement) en dessous de la droite correspondant à l'ordre 4, et l'ordre étant l'entier le plus grand entier vérifiant cette propriété, on déduit que l'ordre est bien 4

## Tableau de Butcher 

**Bonus : Implémenter une méthode de Runge-Kutta générale explicite utilisant le tableau de Butcher. On implementera une fonction update_rk qui retourne $y_{n+1}$ étant données $f,t_n,y_n,h$ et les tableaux $A,B,C$ stockant respectivement les coefficients $(a_{ij})_{1 \leq i,j \leq p}$, $(b_j)_{j=1,...,p}$ et $(c_i)_{i=1,...,p}$ décrits dans le sujet de la PC.**

In [308]:
def update_rk(f, t, y, h, A, B, C):
    """retourne l'approximation yn+1 de y au temps tn+1 = tn + h avec une méthode de Runge-Kutta 
    
    Parameters
    ----------
    f : fonction f(t,y) second membre de l'EDO
    t : temps tn
    y : valeur de yn
    h : pas de temps
    A : matrice A du tableau de Butcher (incluant les zéros)
    B : vecteur B du tableau de Butcher 
    C : vecteur C du tableau de Butcher (incluant c1 = 0)
    
    Returns
    -------
    ynp1 : valeur de yn+1
    """
    p = len(A)
    k = np.zeros(p)
    
    # Calcul de k_i
    for i in range(1,p+1) :
        s = 0
        for j in range(1,i) :
            s += A[i-1][j-1]*k[j-1]
        k[i-1] = f(t+C[i-1]*h, y + h*s)
    
    s = 0
    for j in range(p) :
        s += B[j]*k[j]
    ynp1 = y + h*s
    
    return ynp1


def rk(f, tini, tend, nt, yini, A, B, C):
    """retourne le vecteur contenant les valeurs des yn en appelant update_rk
       
    
    Parameters
    ----------
    f : fonction f(t,y) second membre de l'EDO
    tini : temps initial
    tend : temps final
    nt : dimension du vecteur de sortie
    yini : valeur initiale y(tini)
    A : matrice A du tableau de Butcher (incluant les zéros)
    B : vecteur B du tableau de Butcher 
    C : vecteur C du tableau de Butcher (incluant c1 = 0)

    
    Returns
    -------
    y : vecteur les nt valeurs yn
    """

    t = np.linspace(tini, tend, nt)
    y = np.zeros(nt)
    y[0] = yini
    h = t[1]-t[0]
    
    for n in range(1,nt) :
        y[n] = update_rk(f,t[n-1],y[n-1],h,A,B,C)
    
    return y

**Bonus : Tester avec les tableaux de Butcher correspondant aux méthodes rk2 et rk4 déjà implémentés. Vérifier que vous obtenez les mêmes résultats sur le cas $f(t,y)=y$.**

In [309]:
### RK2 en utilisant la fonction rk

tini = 0
tend = 1
yini = 1
nt = 10

A = np.array([ [0,0],[1/2,0] ])
B = np.array([0,1])
C = np.array([0,1/2])



# Approximation numérique de la solution de l'EDO en utilisant la méthode de RK2
y = rk(f, tini, tend, nt, yini, A, B, C)
t = np.linspace(tini, tend, nt)

# La fonction exacte solution de l'EDO
tsol = np.linspace(tini, tend, 200)
ysol = np.exp(tsol)

# Figure
fig = figure(width=490, height=300)
fig.line(t,y, legend = "y_RK2")
fig.line(tsol,ysol, color="orange", legend = "exp(t)")
fig.legend.location = "top_left"


################## Erreur
# le tableau "entiers" contient les valeurs de nt, et E l'erreur de convergence associées à chaque valeur
N = 200
ntini =10
pas = 10

entiers = np.array([n for n in range(ntini, N, pas)])
E = np.zeros(len(entiers))

# Remplissage de E
for i in range(len(entiers)) :
    nt = entiers[i]
    y = rk(f, tini, tend, nt, yini, A, B, C)
    t = np.linspace(tini, tend, nt)
    ysol = np.exp(t)
    err = abs(ysol-y)
    E[i] = max(err)

# Figure
fig2 = figure(width=490, height=300, x_axis_type="log", y_axis_type="log")
C1 = E[0]*entiers[0];  C2 = E[0]*entiers[0]**2;  C3 = E[0]*entiers[0]**3
fig2.line(entiers,C1/entiers, color="yellow", legend="ordre 1")
fig2.line(entiers,C2/entiers**2, color="orange",legend="ordre 2")
fig2.line(entiers,C3/entiers**3, color="pink", legend="ordre 3")
fig2.x(entiers,E, legend = "max|y(t)-yn|_RK2")
fig2.legend.location = "bottom_left"


show(row(fig,fig2))

In [310]:
### RK2 en utilisant la fonction rk

tini = 0
tend = 1
yini = 1
nt = 20

A = np.array([ [0,0,0,0],[1/2,0,0,0],[0,1/2,0,0],[0,0,1,0] ])
B = np.array([1/6, 1/3, 1/3, 1/6])
C = np.array([0,1/2,1/2,1])


# Approximation numérique de la solution de l'EDO en utilisant la méthode de RK4
y = rk(f, tini, tend, nt, yini, A, B, C)
t = np.linspace(tini, tend, nt)

# La fonction exacte solution de l'EDO
tsol = np.linspace(tini, tend, 200)
ysol = np.exp(tsol)

# Figure
fig = figure(width=490, height=300)
fig.line(t,y, legend = "y_RK2")
fig.line(tsol,ysol, color="orange", legend = "exp(t)")
fig.legend.location = "top_left"


################## Erreur
# le tableau "entiers" contient les valeurs de nt, et E l'erreur de convergence associées à chaque valeur
N = 100
ntini =10
pas = 10

entiers = np.array([n for n in range(ntini, N, pas)])
E = np.zeros(len(entiers))

# Remplissage de E
for i in range(len(entiers)) :
    nt = entiers[i]
    y = rk(f, tini, tend, nt, yini, A, B, C)
    t = np.linspace(tini, tend, nt)
    ysol = np.exp(t)
    err = abs(ysol-y)
    E[i] = max(err)

# Figure
fig2 = figure(width=490, height=300, x_axis_type="log", y_axis_type="log")
C1 = E[0]*entiers[0]**3;  C2 = E[0]*entiers[0]**4;  C3 = E[0]*entiers[0]**5
fig2.line(entiers,C1/entiers**3, color="yellow", legend="ordre 3")
fig2.line(entiers,C2/entiers**4, color="orange",legend="ordre 4")
fig2.line(entiers,C3/entiers**5, color="pink", legend="ordre 5")
fig2.x(entiers,E, legend = "max|y(t)-yn|_RK4")
fig2.legend.location = "bottom_left"

show(row(fig,fig2))

**Commentaire :**  
En définissant correctement A,B et C, on obtient les mêmes résultats que précédemment.

# Un système conservatif

On considère l'ocillateur harmonique :
\begin{equation}
\left\{
\begin{aligned}
q'&=p\\
p'&=-q.
\end{aligned}
\right.
\end{equation}

## Schéma d'Euler explicite et implicite

**Implémenter les schémas d'Euler explicite et implicite pour le système précédent.**

Commencons par implémenter la solution analytique du système 2

In [311]:
def sol2(t, qini, pini) :
    tini = t[0]
    a = np.cos(tini)*qini - np.sin(tini)*pini
    b = np.sin(tini)*qini + np.cos(tini)*pini
    
    Cos = np.cos(t)
    Sin = np.sin(t)
    
    q = a*Cos + b*Sin
    p = -a*Sin + b*Cos
    
    return q, p

In [312]:
"""
La fonction euler_explicite/implicite retourne le couple (qnp1, pnp1) à l'instant t_{n+1}
Paramètres :
_ q = qn
_ p = pn
_ h = pas de temps
"""
# Euler explicite
def update_Eexplicite(q, p, h):
    qnp1 = q + h*p
    pnp1 = p - h*q
    return qnp1, pnp1

def euler_explicite(pini, qini, yini, nt, h) :
    qe = np.zeros(nt)
    pe = np.zeros(nt)
    qe[0], pe[0] = qini, pini
    for n in range(1,nt) :
        qe[n], pe[n] = update_Eexplicite(qe[n-1], pe[n-1], h)
    return qe, pe
    

# Euler implicite
def update_Eimplicite(p, q, h):
    qnp1 = (-h*q + p)/(1+h*h)
    pnp1 = (q + h*p)/(1+h*h)
    return qnp1, pnp1

def euler_implicite(pini, qini, yini, nt, h) :
    qi = np.zeros(nt)
    pi = np.zeros(nt)
    qi[0], pi[0] = qini, pini
    for n in range(1,nt) :
        qi[n], pi[n] = update_Eimplicite(qi[n-1], pi[n-1], h)
    return qi, pi
    

**Représenter les solutions $q_n$ et $p_n$ obtenues, ainsi que l'hamiltonien $\mathcal{H}(q_n,p_n)$ associé. On pourra tracer $q$ et $p$ en fonction du temps, mais aussi tracer la solution dans le plan de phase $(q,p)$. Commenter les résultats obtenus.**

In [315]:
tini = 0
tend = 40
nt = 1000
qini = 1
pini = 0

t = np.linspace(tini, tend, nt)
h = t[1]-t[0]
qsol, psol = sol2(t, qini, pini)

# Euler explicite
qe, pe = euler_explicite(pini, qini, yini, nt, h)
He = (qe*qe+pe*pe)/2


# Euler implicite
qi, pi = euler_implicite(pini, qini, yini, nt, h)
Hi = (qi*qi+pi*pi)/2


# Figure
fig = figure(width=490, height=300)
fig.line(qe, pe, legend = "euler_explicite")
fig.line(qsol, psol, color = "orange", legend = "solution analytique")
fig.x(qe[0],pe[0], color = "red", size = 10)
fig.legend.location = "top_left"
fig2 = figure(width=490, height=300)
fig2.line(t, He, legend ="Energie_explicite")
fig2.legend.location = "top_left"

figb = figure(width=490, height=300)
figb.line(qi, pi, legend = "euler_implicite")
figb.line(qsol, psol, color = "orange", legend = "solution analytique")
figb.x(qi[0],pi[0], color = "red", size = 10)
figb.legend.location = "top_left"
figb2 = figure(width=490, height=300)
figb2.line(t, Hi, legend ="Energie_implicite")


grid = gridplot([[fig, fig2], [figb, figb2]])
show(grid)

**Commentaire :**
Théoriqement, l'énergie $H(q,p)$ est conservée, elle ne varie pas avec le temps, et par conséquent $\forall t, (p(t)^2+q(t)^2)/2 = H_0$ où $H_0$ est la valeur initiale de l'énergie. Ceci montre que dans le plan de phase, l'ensemble des points $(q(t),p(t))$ forment une ellipse.  
Or, en appliquant le shéma d'Euler implicite :
$$q_{n+1} = q_n + hp_n$$ $$p_{n+1} = p_n - hq_n$$ Et on peut montrer par récurrence que $$H(q_n,p_n) = H_0(1+h)^n$$
L'energie obtenue diverge avec le temps, c'est bien ce qu'on remarque sur la figure obtenue. Et on remarque aussi que $(q_n,p_n)$ est poussé vers l'extérieur au cours du temps.  
Par contre, pour le shéma d'Euler implicite :
$$q_{n+1} = (q_n + hp_n)/(1+h^2)$$ $$p_{n+1} = (p_n - hq_n)/(1+h^2)$$ Et on peut montrer par récurrence que $$H(q_n,p_n) = H_0/(1+h)^n$$
L'energie cette fois tend vers 0, ce qui correspond à une atténuation de $q$, et $(q_n,p_n)$ est attiré au centre au cours du temps.  

**Conclusion :** Aucun des deux shémas ne donne une approximation exacte de la solution de l'equation différentielle, toutefois, en augmentant assez la taille de la subdivision par rapport à la longueur de $[t_{ini},t_{end}]$, on peut obtenir une approximation aussi bonne que l'on souhaite. (par exemple pour tini = 0, tend = 10 et n = 5000, l'erreur est majorée par 0.1)  
La solution recherchée est périodique de période $2\pi$, on peut donc se contenter de prendre par exemple tini=0, tend=6.3 (6.3>2$\pi$)  

## Schéma d'ordre élevé

**Appliquer le schéma RK2 (décrit dans l'exercice précédent) au système de l'ocillateur harmonique. Comparer les résultats $p_n$, $q_n$ et $\mathcal{H}(q_n,p_n)$ avec ceux obtenus à la question précédente. Commenter les résultats obtenus.**

In [317]:
tini = 0
tend = 10
nt = 100
qini = 1
pini = 0

yini = np.array([qini, pini])
def f(t,y) :
    return np.array([y[1], -y[0]])

t = np.linspace(tini, tend, nt)

# Choisir RK2 ou RK4 -------------------------------------------
y = rk2(f, tini, tend, nt, yini)
#y = rk4(f, tini, tend, nt, yini)
#---------------------------------------------------------------

q = np.array([y[i][0] for i in range(nt)])
p = np.array([y[i][1] for i in range(nt)])
H = (p*p + q*q)/2

fig = figure(width=490, height=300)
fig.line(q, p, legend = "(q,p)")
fig.x(q[0],p[0], color = "red", size = 10)
fig.legend.location = "top_left"
fig2 = figure(width=490, height=300)
fig2.line(t, H, legend ="Energie")

show(row(fig,fig2))

**Commentaire :**  
L'equation différentielle vérifiée par $q$ peut se ramener à $y' = (y[1], -y[0]) = f(t,y) $ avec $y = (q,p)$.  
J'ai adapté la fonction rk2 définie dans l'exercice 2 pour qu'elle puisse agir également sur des vecteurs.
On remarque que la méthode de Runge-Kutta d'ordre 2 donne une meilleure approximation de la solution de l'EDP, car pour un rapport np/(tend-tini) = 10, on obtient une energie qui ne varie que très peu, et une solution très acceptable.  
J'ai laissé dans mon code aussi la possibilité d'utiliser la méthode de RK-4, qui donne approximation encore meilleure que RK-2

## Schéma symplectique

Considérez le schéma suivant :

\begin{equation*}
  \label{schema_symp}
  y_{n+1} = y_n + h f\left(t_n+\frac{h}{2},\ \frac{y_n+y_{n+1}}{2}\right).
\end{equation*}

**Implémenter ce schéma pour le système  l'ocillateur harmonique.**

In [318]:
def update_pt_milieu_symplectique(q, p, h):
    qnp1 = ( (1-h*h/4)*q + h*p )/(1+h*h/4)
    pnp1 = ( (1-h*h/4)*p - h*q )/(1+h*h/4)
    return qnp1, pnp1

def pt_milieu_symplectique(pini, qini, yini, nt, h):
    q = np.zeros(nt)
    p = np.zeros(nt)
    q[0], p[0] = qini, pini
    for n in range(1,nt) :
        q[n], p[n] = update_pt_milieu_symplectique(q[n-1], p[n-1], h)
    return q, p

**Observer numériquement son ordre, ainsi que l'évolution de l'hamiltonien, et commenter les résultats obtenus.**

In [319]:
tini = 0
tend = 10
nt = 100
qini = 1
pini = 0

t = np.linspace(tini, tend, nt)
h = t[1]-t[0]

# Euler explicite
q, p = pt_milieu_symplectique(pini, qini, yini, nt, h)
H = (q*q+p*p)/2

# Figure
fig = figure(width=490, height=300)
fig.line(q, p, legend = "euler_explicite")
fig.x(q[0],p[0], color = "red", size = 10)
fig.legend.location = "top_left"
fig2 = figure(width=490, height=300)
fig2.line(t, H, legend ="Energie_explicite")
fig2.legend.location = "top_left"

show(row(fig,fig2))

In [299]:
# le tableau "entiers" contient les valeurs de nt, et E l'erreur de convergence associées à chaque valeur
N = 100
ntini =10
pas = 10
entiers = np.array([n for n in range(ntini, N, pas)])

# Erreur Euler_explicite --------------------------------------------
E = np.zeros(len(entiers))
# Remplissage de E
for i in range(len(entiers)) :
    nt = entiers[i]
    t = np.linspace(tini, tend, nt)
    h = t[1] - t[0]
    q, p = pt_milieu_symplectique(pini, qini, yini, nt, h)
    qsol, psol = sol2(t, qini, pini)
    err = abs(q-qsol)
    E[i] = max(err)

# Figure
fig = figure(width=490, height=300, x_axis_type="log", y_axis_type="log")
C1 = E[0]*entiers[0];  C2 = E[0]*entiers[0]**2;  C3 = E[0]*entiers[0]**3
fig.line(entiers,C1/entiers, color="yellow", legend="ordre 1")
fig.line(entiers,C2/entiers**2, color="orange",legend="ordre 2")
fig.line(entiers,C3/entiers**3, color="pink", legend="ordre 3")
fig.x(entiers,E, legend = "erreur pour euler_explicite")
fig.legend.location = "bottom_left"


show(fig)

**Commentaire :**  
La méthode du pont de milieu simplectique est donc d'ordre 2.  